In [ ]:
# import python packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pickle
import os
import importlib
#from scipy.stats import chi2_contingency
#from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import itertools
#from scipy import stats

In [ ]:
site_list = ['KUMC', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC']
ext_list = ['csv','dsv', 'dsv', 'csv', 'csv', 'csv', 'csv', 'csv', 'csv']
sep_list = [',','|', '|', '|', ',', ',', ',', ',', '|']
encoding_list = ['utf-8','utf-8','utf-8','utf-8','utf-8','utf-8', 'windows-1252', 'utf-8','utf-16'] 
ct = 0

daily_average = True


site = site_list[ct]
ext = ext_list[ct]
sep = sep_list[ct]
encoding = encoding_list[ct]
path = []

if site != 'KUMC':
    rawpath = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/' + site + '/raw/'
else: 
    rawpath = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/' + site + '_ORCALE/raw/'
path.append(rawpath)
path.append('/blue/yonghui.wu/hoyinchan/Data/data2022/' + site + '/')
pdata = '/blue/yonghui.wu/hoyinchan/Data/data2022/'+ site 
site

In [ ]:
def try_load_csv(filename, ext, sep, path):
    filename = filename.replace('AKI_', '').lower()
    df = pd.read_parquet(path[1] + 'p0_' + filename + '_' + site +'.parquet')
    df['PATID'] = df['PATID'].astype(str)
    df['ENCOUNTERID'] = df['ENCOUNTERID'].astype(str)    
    return df


def try_load_csv_old(filename, ext, sep, path):
    if 'UMHC' in path[0]:
        filename = 'DEID_'+filename    
        
    try:
        # Try to load the file from the first path
        df = pd.read_csv( path[0] +  filename + '.' + ext, engine="pyarrow", sep=sep, encoding=encoding)
    except Exception as e:
        print(f"Failed to load from {path[0]}: {e}. Loading converted csv...")
        try:
            # If the first attempt fails, try to load the file from the second path
            #df = pd.read_csv( path[0] +  filename + '.' + ext, engine="pyarrow", sep=sep)
            if 'UofU' in path[0]: 
                df = pd.read_csv(path[0] + filename + '.' + ext,  sep=sep, on_bad_lines = 'skip') #pd.read_csv(path[1] + filename + '.csv', engine="pyarrow", sep=',')
            else:
                df = pd.read_csv(path[1] +  filename + '.csv', engine="pyarrow", sep=',', encoding=encoding)
        except Exception as e:
            try:
                # If the first attempt fails, try to load the file from the second path
                if 'UofU' in path[0]: 
                    df = pd.read_csv(path[0] +  filename + '.csv', sep=sep, on_bad_lines = 'skip', encoding=encoding)
                else:                
                    df = pd.read_csv(path[1] +  filename + '.csv', sep=',', on_bad_lines = 'skip', encoding=encoding)
            except Exception as e:
                # If the second attempt also fails, handle or re-raise the exception
                print(f"Failed to load from {path[1]} as well: {e}")
                raise Exception(f"Could not load the file from either path.")
    
    return df

In [ ]:
def pat_count(df, var = 'PATID'):
    return df[var].nunique()

def null_count(data):
    return data.isnull().sum()/data.shape[0]

def enc_count(df):
    return df['ENCOUNTERID'].nunique()

def aki_count(df):
    return ( [pat_count(df), enc_count(df)])

In [ ]:
def read_onset_data(path, ext, sep):
    ## AKI staging
    xxx = try_load_csv_old('AKI_LAB_SCR', ext, sep, path) #pd.read_csv(path+"AKI_LAB_SCR."+ext, engine="pyarrow", sep=sep)
    yyy = try_load_csv_old('AKI_ONSETS', ext, sep, path) #pd.read_csv(path+"AKI_ONSETS."+ext, engine="pyarrow", sep=sep)
    xxx.columns = xxx.columns.str.upper()
    yyy.columns = yyy.columns.str.upper()
    xxx.rename(columns = {'SPECIMEN_DATE"+PD.DATE_SHIFT"': 'SPECIMEN_DATE'}, inplace = True)
    yyy.rename(columns = {'ADMIT_DATE"+PD.DATE_SHIFT"': 'ADMIT_DATE'}, inplace = True)
    yyy = yyy[['ENCOUNTERID', 'PATID', 'ADMIT_DATE']]
    yyy.columns = ['ENCOUNTERID', 'PATID', 'ADMIT_DATE']
    
#    xxx = xxx[['ONSETS_ENCOUNTERID', 'PATID', 'SPECIMEN_DATE', 'SPECIMEN_TIME', 'RESULT_NUM', 'DAYS_SINCE_ADMIT']]
#    xxx.columns = ['ENCOUNTERID', 'PATID', 'SPECIMEN_DATE', 'SPECIMEN_TIME', 'RESULT_NUM', 'DAYS_SINCE_ADMIT']

    xxx = xxx[['ONSETS_ENCOUNTERID', 'PATID', 'SPECIMEN_DATE', 'RESULT_NUM', 'DAYS_SINCE_ADMIT']]
    xxx.columns = ['ENCOUNTERID', 'PATID', 'SPECIMEN_DATE', 'RESULT_NUM', 'DAYS_SINCE_ADMIT']

    xxx = xxx.merge(yyy, on = ['ENCOUNTERID', 'PATID'], how='left')
    #xxx['SPECIMEN_TIME'] = xxx['SPECIMEN_TIME'].fillna('00:00:00')
    xxx = xxx.dropna()
    xxx['SPECIMEN_DATE'] = pd.to_datetime(pd.to_datetime(xxx['SPECIMEN_DATE']).dt.date)
    # xxx['SPECIMEN_DATETIME'] = xxx['SPECIMEN_DATE'].astype(str)+' '+xxx['SPECIMEN_TIME'].astype(str)
    # xxx['SPECIMEN_DATETIME'] = pd.to_datetime(xxx['SPECIMEN_DATETIME'])
    
    xxx['ADMIT_DATE'] = pd.to_datetime(pd.to_datetime(xxx['ADMIT_DATE']).dt.date)
    xxx['DAYS_SINCE_ADMIT'] = (xxx['SPECIMEN_DATE']-xxx['ADMIT_DATE']).dt.days
    # xxx = xxx.sort_values('SPECIMEN_DATE')
    # xxx = xxx[['PATID', 'ENCOUNTERID', 'SPECIMEN_DATETIME', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['PATID', 'ENCOUNTERID', 'SPECIMEN_DATETIME', 'DAYS_SINCE_ADMIT']).mean()
    xxx = xxx.sort_values(['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE'])
    xxx = xxx.reset_index()
    return xxx.drop_duplicates(), yyy.drop_duplicates()

In [ ]:
df_onset, df_y = read_onset_data(path, ext, sep)

df_onsetw2i = df_onset.copy()
df_onsetw2i['RESULT_NUM'] = df_onsetw2i['RESULT_NUM']*10000
df_onsetw2i['RESULT_NUM'] = df_onsetw2i['RESULT_NUM'].astype(int)

if daily_average:
    df_onset = df_onset.groupby(['ENCOUNTERID', 'PATID', 'ADMIT_DATE', 'SPECIMEN_DATE', 'DAYS_SINCE_ADMIT']).mean().reset_index()

if daily_average:
    df_onsetw2i = df_onsetw2i.groupby(['ENCOUNTERID', 'PATID', 'ADMIT_DATE', 'SPECIMEN_DATE', 'DAYS_SINCE_ADMIT']).mean().reset_index()

df_onsetw2i['RESULT_NUM'] = df_onsetw2i['RESULT_NUM']/10000.0

# df_onsetw2i['INT_RESULT_NUM'] = df_onsetw2i['RESULT_NUM']
# df_onsetw2i = df_onsetw2i.drop('RESULT_NUM',axis=1)

# df_onsetw3i = df_onset.merge(df_onsetw2i, on = ['ENCOUNTERID', 'PATID', 'ADMIT_DATE', 'SPECIMEN_DATE', 'DAYS_SINCE_ADMIT'], how='outer')

# df_onsetw4i = df_onsetw3i[df_onsetw3i['RESULT_NUM']!=df_onsetw3i['INT_RESULT_NUM']]

# for i in range(df_onsetw4i.shape[0]):
#     print('---')
#     print(df_onsetw4i['RESULT_NUM'].iloc[i])
#     print(df_onsetw4i['INT_RESULT_NUM'].iloc[i])

#df_onset = df_onsetw2i

In [ ]:
df_onset.to_pickle(pdata+'/scr00.pkl')
df_y.to_pickle(pdata+'/admit00.pkl')

In [ ]:
# df_onset.to_pickle(pdata+'/scr00.pkl')
# df_y.to_pickle(pdata+'/admit00.pkl')

In [ ]:
# df_onset = pd.read_pickle(pdata+'/scr00.pkl')
# df_y = pd.read_pickle(pdata+'/admit00.pkl')

In [ ]:
df_y['PATID'] = df_y['PATID'].astype(str)
df_y['ENCOUNTERID'] = df_y['ENCOUNTERID'].astype(str)
df_onset['PATID'] = df_onset['PATID'].astype(str)
df_onset['ENCOUNTERID'] = df_onset['ENCOUNTERID'].astype(str)

In [ ]:
def inverse_MDRD(row, KDIGO_baseline):
    age = row["AGE"]
    is_male = True if row["MALE"]  else False
    is_black = True if row["RACE_BLACK"] else False
        
    if is_male and is_black:
        if age <= 24:
            return KDIGO_baseline.loc["20-24", "Black males"]
        elif 25 <= age <= 29:
            return KDIGO_baseline.loc["25-29", "Black males"]
        elif 30 <= age <= 39:
            return KDIGO_baseline.loc["30-39", "Black males"]
        elif 40 <= age <= 54:
            return KDIGO_baseline.loc["40-54", "Black males"]
        elif 55 <= age <= 65:
            return KDIGO_baseline.loc["55-65", "Black males"]
        elif age > 65:
            return KDIGO_baseline.loc[">65", "Black males"]
    
    if is_male and not is_black:
        if age <= 24:
            return KDIGO_baseline.loc["20-24", "Other males"]
        elif 25 <= age <= 29:
            return KDIGO_baseline.loc["25-29", "Other males"]
        elif 30 <= age <= 39:
            return KDIGO_baseline.loc["30-39", "Other males"]
        elif 40 <= age <= 54:
            return KDIGO_baseline.loc["40-54", "Other males"]
        elif 55 <= age <= 65:
            return KDIGO_baseline.loc["55-65", "Other males"]
        elif age > 65:
            return KDIGO_baseline.loc[">65", "Other males"]

    if not is_male and is_black:
        if age <= 24:
            return KDIGO_baseline.loc["20-24", "Black females"]
        elif 25 <= age <= 29:
            return KDIGO_baseline.loc["25-29", "Black females"]
        elif 30 <= age <= 39:
            return KDIGO_baseline.loc["30-39", "Black females"]
        elif 40 <= age <= 54:
            return KDIGO_baseline.loc["40-54", "Black females"]
        elif 55 <= age <= 65:
            return KDIGO_baseline.loc["55-65", "Black females"]
        elif age > 65:
            return KDIGO_baseline.loc[">65", "Black females"]
    
    if not is_male and not is_black:
        if age <= 24:
            return KDIGO_baseline.loc["20-24", "Other females"]
        elif 25 <= age <= 29:
            return KDIGO_baseline.loc["25-29", "Other females"]
        elif 30 <= age <= 39:
            return KDIGO_baseline.loc["30-39", "Other females"]
        elif 40 <= age <= 54:
            return KDIGO_baseline.loc["40-54", "Other females"]
        elif 55 <= age <= 65:
            return KDIGO_baseline.loc["55-65", "Other females"]
        elif age > 65:
            return KDIGO_baseline.loc[">65", "Other females"]

In [ ]:
def get_scr_baseline_old(path, df_onset, df_y, ext, sep):

    # process dx data
    dx = try_load_csv_old('AKI_DX', ext, sep, path) # pd.read_csv(path+"AKI_DX."+ext, sep= sep, engine="pyarrow")
    print(dx.shape[0])
    dx.columns = dx.columns.str.upper()
    dx.rename(columns = {'DX_DATE"+PD.DATE_SHIFT"': 'DX_DATE'}, inplace = True)
    dx["ENCOUNTERID"] = dx["ONSETS_ENCOUNTERID"]
    dx['ENCOUNTERID'] = dx['ENCOUNTERID'].astype(str)
    dx['PATID'] = dx['PATID'].astype(str)
    dx = dx[["PATID", "ENCOUNTERID", "DX", "DAYS_SINCE_ADMIT", "DX_DATE", "DX_TYPE"]]
    print(dx.shape[0])
    
    dx = df_y[["PATID","ENCOUNTERID", 'ADMIT_DATE']].merge(dx, on = ["PATID","ENCOUNTERID"], how = "inner")
    print(dx.shape[0])

    dx['ADMIT_DATE'] = pd.to_datetime(pd.to_datetime(dx['ADMIT_DATE']).dt.date)
    dx['DX_DATE'] = pd.to_datetime(pd.to_datetime(dx['DX_DATE']).dt.date)
#    dx['DAYS_SINCE_ADMIT'] = (dx['DX_DATE']-dx['ADMIT_DATE']).dt.days

    # process demo data
    demo = try_load_csv('AKI_DEMO', ext, sep, path) # pd.read_csv(path+'AKI_DEMO.'+ext, sep=sep,  engine="pyarrow")
    demo.columns = demo.columns.str.upper()
    demo['ENCOUNTERID'] = demo['ONSETS_ENCOUNTERID']
    demo['ENCOUNTERID'] = demo['ENCOUNTERID'].astype(str)
    demo['PATID'] = demo['PATID'].astype(str)
    demo['MALE'] = demo['SEX'] == 'M'
    demo['HISPANIC'] = demo['HISPANIC'] == 'Y'
    demo['RACE_WHITE'] = demo['RACE'] == '05'
    demo['RACE_BLACK'] = demo['RACE'] == '03'
    demo = demo[['PATID', 'ENCOUNTERID', 'AGE', 'MALE', 'RACE_WHITE', 'RACE_BLACK', 'HISPANIC']]
    demo = demo.drop_duplicates()
    
    return dx, demo

In [ ]:
def get_scr_baseline(path, df_onset, df_y, ext, sep):

    # process dx data
    dx = try_load_csv('AKI_DX', ext, sep, path) # pd.read_csv(path+"AKI_DX."+ext, sep= sep, engine="pyarrow")
    print(dx.shape[0])
    dx.columns = dx.columns.str.upper()
    dx.rename(columns = {'DX_DATE"+PD.DATE_SHIFT"': 'DX_DATE'}, inplace = True)
    dx["ENCOUNTERID"] = dx["ONSETS_ENCOUNTERID"]
    dx['ENCOUNTERID'] = dx['ENCOUNTERID'].astype(str)
    dx['PATID'] = dx['PATID'].astype(str)
    dx = dx[["PATID", "ENCOUNTERID", "DX", "DAYS_SINCE_ADMIT", "DX_DATE", "DX_TYPE"]]
    print(dx.shape[0])
    
    dx = df_y[["PATID","ENCOUNTERID", 'ADMIT_DATE']].merge(dx, on = ["PATID","ENCOUNTERID"], how = "inner")
    print(dx.shape[0])

    dx['ADMIT_DATE'] = pd.to_datetime(pd.to_datetime(dx['ADMIT_DATE']).dt.date)
    dx['DX_DATE'] = pd.to_datetime(pd.to_datetime(dx['DX_DATE']).dt.date)
#    dx['DAYS_SINCE_ADMIT'] = (dx['DX_DATE']-dx['ADMIT_DATE']).dt.days

    # process demo data
    demo = try_load_csv('AKI_DEMO', ext, sep, path) # pd.read_csv(path+'AKI_DEMO.'+ext, sep=sep,  engine="pyarrow")
    demo.columns = demo.columns.str.upper()
    demo['ENCOUNTERID'] = demo['ONSETS_ENCOUNTERID']
    demo['ENCOUNTERID'] = demo['ENCOUNTERID'].astype(str)
    demo['PATID'] = demo['PATID'].astype(str)
    demo['MALE'] = demo['SEX'] == 'M'
    demo['HISPANIC'] = demo['HISPANIC'] == 'Y'
    demo['RACE_WHITE'] = demo['RACE'] == '05'
    demo['RACE_BLACK'] = demo['RACE'] == '03'
    demo = demo[['PATID', 'ENCOUNTERID', 'AGE', 'MALE', 'RACE_WHITE', 'RACE_BLACK', 'HISPANIC']]
    demo = demo.drop_duplicates()
    
    return dx, demo

In [ ]:
# df_scr = df_onset
# df_admit = df_y
# c7day = 'MOST_RECENT'
# c365day = 'AVERAGE'
# cckd = 'DROP'

def get_scr_baseline_new(df_scr, df_admit, dx, demo, c7day = 'MOST_RECENT', c365day = 'AVERAGE', cckd = 'DROP'):
#if True:
    cohort_table = dict()
    
    # load & process dx data
#    dx = pd.read_pickle(filepath_lst[0]+'AKI_DX.pkl')  
    dx = dx[['PATID', 'ENCOUNTERID', 'DX', 'DX_DATE', 'DX_TYPE', 'DAYS_SINCE_ADMIT']] 
    dx = df_admit[['PATID', 'ENCOUNTERID', 'ADMIT_DATE']].merge(dx, on = ['PATID', 'ENCOUNTERID'], how = 'inner')
#    dx['DAYS_SINCE_ADMIT'] = (dx['DX_DATE']-dx['ADMIT_DATE']).dt.days

    # calculate DX_DATE when it is missing
    dx['ADMIT_DATE'] = pd.to_datetime(dx['ADMIT_DATE'])
    dx.loc[dx.DX_DATE.isna(), 'DX_DATE'] =  dx.loc[dx.DX_DATE.isna(), 'ADMIT_DATE'] +  pd.to_timedelta(dx.loc[dx.DX_DATE.isna(), 'DAYS_SINCE_ADMIT'], unit='D')

    dx['DX'] = dx['DX'].astype(str)
    dx['DX_TYPE'] = dx['DX_TYPE'].astype(str)
    dx['DX_TYPE'] = dx['DX_TYPE'].replace('09', '9')
    
    # load & process demo data
#    demo = pd.read_pickle(filepath_lst[0]+'AKI_DEMO'+'.pkl')  
#    demo['MALE'] = demo['SEX'] == 'M'

#    demo['RACE_WHITE'] = demo['RACE'] == '05'
#    demo['RACE_BLACK'] = demo['RACE'] == '03'
    demo = demo[['PATID', 'ENCOUNTERID', 'AGE', 'MALE', 'RACE_WHITE', 'RACE_BLACK']]
    demo['AGE'] = demo['AGE'].astype(int)    
    demo = demo.drop_duplicates()
    
    # estimate SCr Baseline
    pat_id_cols = ['PATID', 'ENCOUNTERID']
    complete_df = df_scr[['ENCOUNTERID', 'PATID', 'ADMIT_DATE', 'SPECIMEN_DATE', 'RESULT_NUM']]
 
    # 1. min between the min of 1-week prior admission SCr and within 24 hour after admission SCr
    # SCr within 24 hour after admission, that is admission day and one day after, get mean
    admission_SCr = complete_df[(complete_df.SPECIMEN_DATE >= complete_df.ADMIT_DATE) & \
                                (complete_df.SPECIMEN_DATE <= (complete_df.ADMIT_DATE + pd.Timedelta(days=1)))].copy()

    # Admission SCr is the mean of all the SCr within 24h admission
    admission_SCr = admission_SCr.groupby(pat_id_cols)['RESULT_NUM'].mean().reset_index()

    admission_SCr.rename(columns = {'RESULT_NUM': 'ADMISSION_SCR'}, inplace = True)

    # merge the ADMISSION_SCR back to the main frame
    complete_df = complete_df.merge(admission_SCr, 
                                    on = pat_id_cols,
                                    how = 'left')

    # SCr within 7 days prior to admission
    one_week_prior_admission = complete_df[(complete_df.SPECIMEN_DATE >= complete_df.ADMIT_DATE - pd.Timedelta(days=7)) & \
                                           (complete_df.SPECIMEN_DATE < complete_df.ADMIT_DATE)].copy()
    one_week_prior_admission = one_week_prior_admission.sort_values(by = ['PATID', 'ENCOUNTERID','SPECIMEN_DATE'])
    
    if c7day == 'MOST_RECENT':
        one_week_prior_admission = one_week_prior_admission.groupby(pat_id_cols)['RESULT_NUM'].last().reset_index()
    else:
        one_week_prior_admission = one_week_prior_admission.groupby(pat_id_cols)["RESULT_NUM"].nanmin().reset_index()
        
    one_week_prior_admission = one_week_prior_admission.rename(columns = {'RESULT_NUM': 'ONE_WEEK_SCR'})

    complete_df = complete_df.merge(one_week_prior_admission, 
                                    on = pat_id_cols,
                                    how = 'left')

    # take the min between one week SCr and admission SCr
    complete_df.loc[complete_df.ONE_WEEK_SCR.notna(), 'BASELINE_EST_1'] = \
                np.nanmin(complete_df.loc[complete_df.ONE_WEEK_SCR.notna(), ['ONE_WEEK_SCR','ADMISSION_SCR']], axis = 1)

    complete_dfe = complete_df.drop(['SPECIMEN_DATE', 'RESULT_NUM'],axis=1).drop_duplicates()
    cohort_table['ONE_WEEK_SCR_YES'] = complete_dfe.ONE_WEEK_SCR.notna().sum()
    cohort_table['ONE_WEEK_SCR_NO'] = complete_dfe.ONE_WEEK_SCR.isna().sum()    
    cohort_table['ONE_WEEK_SCR_ONE_WEEK_SCR'] = (complete_dfe.ONE_WEEK_SCR.notna() & (complete_dfe['ONE_WEEK_SCR']==complete_dfe['BASELINE_EST_1'])).sum()
    cohort_table['ONE_WEEK_SCR_ADMISSION_SCR'] = (complete_dfe.ONE_WEEK_SCR.notna() & (complete_dfe['ONE_WEEK_SCR']!=complete_dfe['BASELINE_EST_1'])).sum()

        
    ori_num_unique_combinations = df_scr.groupby(['PATID', 'ENCOUNTERID']).ngroups
    # get the percentage of encounters that do not have past 7-day records
    criterion1_no_missing = complete_df.loc[complete_df.ONE_WEEK_SCR.notna(), :].groupby(pat_id_cols).ngroups
    criterion1_missing_rate = 1 - criterion1_no_missing / ori_num_unique_combinations

    # 2. pre-admission 365-7 day mean
    # here we only care about SCr measurements within 1 year before hospitalization
    one_year_prior_admission = complete_df[(complete_df.SPECIMEN_DATE < (complete_df.ADMIT_DATE - pd.Timedelta(days=7))) & \
                                     (complete_df.SPECIMEN_DATE >= (complete_df.ADMIT_DATE - pd.Timedelta(days=365.25)))].copy()
    one_year_prior_admission = one_year_prior_admission.sort_values(by = ['PATID', 'ENCOUNTERID','SPECIMEN_DATE'])
    one_year_prior_admission = one_year_prior_admission.loc[:, pat_id_cols + ['RESULT_NUM']]
    
    if c365day == 'AVERAGE':
        one_year_prior_admission = one_year_prior_admission.groupby(pat_id_cols)['RESULT_NUM'].mean().reset_index()
    else:
        one_year_prior_admission = one_year_prior_admission.groupby(pat_id_cols)['RESULT_NUM'].last().reset_index()  # or mean()
    
    one_year_prior_admission.rename(columns = {'RESULT_NUM': 'ONE_YEAR_SCR'}, inplace = True)
    
    complete_df = complete_df.merge(one_year_prior_admission, 
                                    on = pat_id_cols,
                                    how = 'left')
    
    # take the min between one week SCr and admission SCr
    complete_df.loc[complete_df.ONE_YEAR_SCR.notna(), 'BASELINE_EST_2'] = \
                np.nanmin(complete_df.loc[complete_df.ONE_YEAR_SCR.notna(), ['ONE_YEAR_SCR', 'ADMISSION_SCR']], axis = 1)

    # priority 1: 7day SCr, priority 2: one year SCr
    complete_df['BASELINE_NO_INVERT'] = \
                np.where(complete_df['BASELINE_EST_1'].isna(), complete_df['BASELINE_EST_2'], complete_df['BASELINE_EST_1'])

    complete_dfe = complete_df.drop(['SPECIMEN_DATE', 'RESULT_NUM'],axis=1).drop_duplicates()
    cohort_table['ONE_YEAR_SCR_YES'] = (complete_dfe.ONE_WEEK_SCR.isna() & complete_dfe.ONE_YEAR_SCR.notna()).sum()
    cohort_table['ONE_YEAR_SCR_NO'] = (complete_dfe.ONE_WEEK_SCR.isna() & complete_dfe.ONE_YEAR_SCR.isna()).sum()
    cohort_table['ONE_YEAR_SCR_ONE_WEEK_SCR'] = (complete_dfe.ONE_WEEK_SCR.isna() & complete_dfe.ONE_YEAR_SCR.notna() & (complete_dfe['ONE_YEAR_SCR']==complete_dfe['BASELINE_EST_2'])).sum()
    cohort_table['ONE_YEAR_SCR_ADMISSION_SCR'] = (complete_dfe.ONE_WEEK_SCR.isna() & complete_dfe.ONE_YEAR_SCR.notna() & (complete_dfe['ONE_YEAR_SCR']!=complete_dfe['BASELINE_EST_2'])).sum()    
    
    # 3. Invert CKD-EPI (2021) to estimate baseline (only for non-CKD patients)
    # get those encounters for which we need to impute baseline
    pat_to_invert = complete_df.loc[complete_df.BASELINE_NO_INVERT.isna(), pat_id_cols+['ADMIT_DATE', 'ADMISSION_SCR']]
    # one patient one row
    pat_to_invert.drop_duplicates(subset=pat_id_cols, keep='first', inplace = True)


    pat_dx = pat_to_invert.merge(dx.drop(['ENCOUNTERID', 'ADMIT_DATE'], axis = 1), 
                                              on = 'PATID', 
                                              how = 'left')


    # check patients that do not have DX in the database
    #pat_dx.DX_DATE.isna().mean()

    # filter out those DX after admission
    pat_dx = pat_dx[pat_dx.DX_DATE <= pat_dx.ADMIT_DATE]
    
#    pat_dx = pat_dx.merge(pat_to_invert[['PATID', 'ENCOUNTERID']], on = ['PATID', 'ENCOUNTERID'], how = 'outer')
    
    # get the default eGFR for inversion: default to 75 for non-CKD patients, average of eGFR from staging criteria for CKD patients
    pat_dx['DFLT_eGFR'] = 75

    pat_dx.loc[pat_dx['DX'].isin(['585.3', 'N18.3']), 'DFLT_eGFR'] = 90/2
    pat_dx.loc[pat_dx['DX'].isin(['585.4', 'N18.4']), 'DFLT_eGFR'] = 45/2
    pat_dx.loc[pat_dx['DX'].isin(['585.5', 'N18.5']), 'DFLT_eGFR'] = 15/2
#    pat_dx.loc[pat_dx['DX'].isin(['585.6', 'N18.6']), 'DFLT_eGFR'] = 15/2

    pat_def_egfr = pat_dx.groupby(pat_id_cols)['DFLT_eGFR'].min().reset_index()
        
    pat_to_invert= pat_to_invert.merge(pat_def_egfr, on = pat_id_cols, how = 'left')
    pat_to_invert['DFLT_eGFR'] = pat_to_invert['DFLT_eGFR'].fillna(75)

    cohort_table['MDRD_NOCKD'] = (pat_to_invert['DFLT_eGFR'] == 75).sum()
    cohort_table['MDRD_CKD3']  = (pat_to_invert['DFLT_eGFR'] == 90/2).sum()
    cohort_table['MDRD_CKD4']  = (pat_to_invert['DFLT_eGFR'] == 45/2).sum()
    cohort_table['MDRD_CKD5']  = (pat_to_invert['DFLT_eGFR'] == 15/2).sum()
    
    pat_to_invert['KEEPNONCKD'] = pat_to_invert['DFLT_eGFR'] == 75
    
    #pat_to_invert.DFLT_eGFR.value_counts()

    # Backcalculation for patients
    # merge DEMO with pat_to_invert
    pat_to_invert = pat_to_invert.merge(demo, on = pat_id_cols, how = 'left')
    
    KDIGO_baseline = np.array([
        [1.5, 1.3, 1.2, 1.0],
        [1.5, 1.2, 1.1, 1.0],
        [1.4, 1.2, 1.1, 0.9],
        [1.3, 1.1, 1.0, 0.9],
        [1.3, 1.1, 1.0, 0.8],
        [1.2, 1.0, 0.9, 0.8]
    ])
    KDIGO_baseline = pd.DataFrame(KDIGO_baseline, columns = ["Black males", "Other males",
                                                            "Black females", "Other females"],
                                 index = ["20-24", "25-29", "30-39", "40-54", "55-65", ">65"])    
    
    
    # estimate SCr from eGFR
    pat_to_invert.loc[:, 'BASELINE_INVERT'] = pat_to_invert.apply(inverse_MDRD, args = (KDIGO_baseline,), axis = 1) #pat_to_invert.apply(inverse_CKDEPI21, axis = 1)

    # take minimum of inverted SCr and admission SCr
    pat_to_invert['BASELINE_EST_3'] = np.nanmin(pat_to_invert[['ADMISSION_SCR', 'BASELINE_INVERT']], axis = 1)

    # merge back the computation results
    complete_df = complete_df.merge(pat_to_invert[pat_id_cols + ['BASELINE_EST_3', 'KEEPNONCKD']], 
                                    on = pat_id_cols,
                                    how = 'left')

    # replace the old baseline
    complete_df['SERUM_CREAT_BASE'] = np.nanmin(complete_df[['BASELINE_NO_INVERT', 'BASELINE_EST_3']], axis = 1)

    complete_ckd = complete_df[~(complete_df['KEEPNONCKD'].fillna(True) | complete_df['BASELINE_NO_INVERT'].notna())]
        
    if cckd == 'DROP':
        complete_df = complete_df[complete_df['KEEPNONCKD'].fillna(True) | complete_df['BASELINE_NO_INVERT'].notna()]
    complete_df = complete_df.drop('KEEPNONCKD', axis=1)

    
    # drop those still cannot find baseline
    complete_df = complete_df.dropna(subset=['SERUM_CREAT_BASE'])

    return complete_df.drop_duplicates(), cohort_table, complete_ckd

In [ ]:
dx, demo = get_scr_baseline_old(path, df_onset, df_y, ext, sep)
dx.to_pickle(pdata+'/dx00.pkl')
demo.to_pickle(pdata+'/demo00.pkl')

In [ ]:
# dx, demo = get_scr_baseline(path, df_onset, df_y, ext, sep)
# dx.to_pickle(pdata+'/dx00.pkl')
# demo.to_pickle(pdata+'/demo00.pkl')

In [ ]:
# df_onset = pd.read_pickle(pdata+'/scr00.pkl')
# df_y = pd.read_pickle(pdata+'/admit00.pkl')
df_y['PATID'] = df_y['PATID'].astype(str)
df_y['ENCOUNTERID'] = df_y['ENCOUNTERID'].astype(str)
df_onset['PATID'] = df_onset['PATID'].astype(str)
df_onset['ENCOUNTERID'] = df_onset['ENCOUNTERID'].astype(str)
# dx = pd.read_pickle(pdata+'/dx00.pkl')
# demo = pd.read_pickle(pdata+'/demo00.pkl')
dx['PATID'] = dx['PATID'].astype(str)
dx['ENCOUNTERID'] = dx['ENCOUNTERID'].astype(str)
demo['PATID'] = demo['PATID'].astype(str)
demo['ENCOUNTERID'] = demo['ENCOUNTERID'].astype(str)

In [ ]:
dx

In [ ]:
#df_base = get_scr_baseline2(path, df_onset, df_y,  ext, sep, dx, demo)
df_base, cohort_table, complete_ckd = get_scr_baseline_new(df_onset, df_y, dx, demo, c7day = 'MOST_RECENT', c365day = 'AVERAGE', cckd = 'DROP')
df_base.to_pickle(pdata+'/df_base00.pkl')

cohort_table['TOTAL'] = cohort_table['ONE_WEEK_SCR_YES'] + cohort_table['ONE_WEEK_SCR_NO']
cohort_table['site'] = site
site_column = cohort_table.pop('site')
cohort_table_df = pd.DataFrame(list(cohort_table.items()), columns=['Category', site_column])
cohort_table_df.index = cohort_table_df['Category']
cohort_table_df = cohort_table_df.drop('Category', axis=1)
cohort_table_df.to_pickle(pdata+'/cohort_table00.pkl')

In [ ]:
cohort_table_df = pd.read_pickle(pdata+'/cohort_table00.pkl')
cohort_table_df.loc['MDRD_CKD3',]+cohort_table_df.loc['MDRD_CKD4',]+cohort_table_df.loc['MDRD_CKD5',]

In [ ]:
cohort_table_df.loc[['TOTAL','ONE_WEEK_SCR_YES', 'ONE_WEEK_SCR_NO', 'ONE_YEAR_SCR_YES', 'ONE_YEAR_SCR_NO', 'MDRD_NOCKD']]

In [ ]:
def get_rrt(path, ext, sep, df_y):
    px = try_load_csv('AKI_PX', ext, sep, path) # pd.read_csv(path+"AKI_PX."+ext, engine="pyarrow", sep= sep)
    px.columns = px.columns.str.upper()
    px.rename(columns = {'PX_DATE"+PD.DATE_SHIFT"': 'PX_DATE'}, inplace = True)
    px["ENCOUNTERID"] = px["ONSETS_ENCOUNTERID"]
    px['PATID'] = px['PATID'].astype(str)
    px['ENCOUNTERID'] = px['ENCOUNTERID'].astype(str)
    
    
    idx_transplant = np.logical_or(np.logical_or(
                           np.logical_and(px['PX_TYPE']=='CH',px['PX'].isin(['50300','50320','50323','50325','50327','50328','50329','50340','50360','50365','50370','50380'])),
                           np.logical_and(px['PX_TYPE']=='09',px['PX'].isin(['55.51','55.52','55.53','55.54','55.61','55.69']))),np.logical_or(
                           np.logical_and(px['PX_TYPE']=='9',px['PX'].isin(['55.51','55.52','55.53','55.54','55.61','55.69'])),                       
                           np.logical_and(px['PX_TYPE']=='10',px['PX'].isin(['0TY00Z0','0TY00Z1','0TY00Z2','0TY10Z0','0TY10Z1','0TY10Z2','0TB00ZZ','0TB10ZZ','0TT00ZZ','0TT10ZZ','0TT20ZZ']))))

    idx_dialysis =( ( (px['PX_TYPE']=='CH')  &  (px['PX'].isin(['90935', '90937'])))  |   # ( (px['PX_TYPE']=='CH')  & 
        ( (px['PX_TYPE']=='CH')  & (pd.to_numeric(px['PX'], errors='coerce').between(90940, 90999))) |   #(px['PX_TYPE']=='CH')  &
        ( (px['PX_TYPE']=='9')  & ( (px['PX'].isin(['39.93','39.95','54.98', 'V45.11']))) )  |  #(px['PX_TYPE']=='9')  &
        ( (px['PX_TYPE']=='09')  & (px['PX'].isin(['39.93','39.95','54.98', 'V45.11'])) ) |  #(px['PX_TYPE']=='09')  &
        ( (px['PX_TYPE']=='10')  & (px['PX'].isin(['5A1D00Z','5A1D60Z','5A1D70Z','5A1D80Z','5A1D90Z', 'Z99.2']))) )  #(px['PX_TYPE']=='10')  &
   
 
    rrt_stage =  px[idx_transplant | idx_dialysis] 

    
    rrt_stage['PX_DATE'] = pd.to_datetime(pd.to_datetime(rrt_stage['PX_DATE']).dt.date )
    df_y['ADMIT_DATE'] = pd.to_datetime(pd.to_datetime(df_y['ADMIT_DATE']).dt.date ) 
    rrt_stage = rrt_stage[['PATID','ENCOUNTERID','PX_DATE']]
    rrt_stage.columns = ['PATID','ENCOUNTERID','RRT3_ONSET_DATE']
    rrt_stage['RRT3_ONSET_DATE'] = pd.to_datetime(pd.to_datetime(rrt_stage['RRT3_ONSET_DATE']).dt.date )

    rrt_stage = rrt_stage.merge(df_y, on=['PATID', 'ENCOUNTERID'], how='inner')
    
    rrt_stage['RRT3_ONSET_DATE'] = pd.to_datetime(rrt_stage['RRT3_ONSET_DATE'])
    rrt_stage['ADMIT_DATE'] = pd.to_datetime(rrt_stage['ADMIT_DATE'])
    rrt_stage['RRT3_SINCE_ADMIT'] = (rrt_stage['RRT3_ONSET_DATE']-rrt_stage['ADMIT_DATE']).dt.total_seconds()/(3600*24)
    rrt_stage = rrt_stage.loc[rrt_stage[['ENCOUNTERID', 'RRT3_SINCE_ADMIT']].groupby('ENCOUNTERID').idxmin().reset_index()['RRT3_SINCE_ADMIT']]
    rrt_stage.drop('ADMIT_DATE', axis = 1, inplace = True)
    return rrt_stage, px

In [ ]:
#df_rrt, px = get_rrt(path, ext, sep, df_y)
#df_rrt.to_pickle(pdata+'/df_rrt00.pkl')
#px.to_pickle(pdata+'/px.pkl')

df_rrt = pd.read_pickle(pdata+'/df_rrt00.pkl')
px = pd.read_pickle(pdata+'/px.pkl')
#df_rrt

In [ ]:
def read_onset_new(df_onset, df_y, df_rrt, df_base, part = 'either'):
    xxx = df_onset.copy()
    yyy = df_y.copy()

    zzz = df_base[['PATID', 'ENCOUNTERID', 'SERUM_CREAT_BASE']].drop_duplicates()
    
    # zzz['SERUM_CREAT_BASE'] = zzz['SERUM_CREAT_BASE'].round(4)
    # xxx['RESULT_NUM'] = xxx['RESULT_NUM'].round(4)
    
    zzz.columns= ['PATID', 'ENCOUNTERID',  'RESULT_NUM_BASE_7d']
    xxx = xxx.merge(zzz, on = ['PATID', 'ENCOUNTERID'], how='inner')

    zzz2 = xxx[['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE', 'RESULT_NUM']].groupby(['PATID', 'ENCOUNTERID']).rolling('2d', on='SPECIMEN_DATE').min().reset_index()
    zzz2 = zzz2[['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE', 'RESULT_NUM']]
    zzz2.columns= ['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE', 'RESULT_NUM_BASE_2d']
    xxx = xxx.merge(zzz2, on = ['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE'], how='left')

    #0.3 increase in 48 hours
    #1.5 increase in 7 days
    xxx['AKI1.5'] = (xxx['RESULT_NUM']>=1.5*xxx['RESULT_NUM_BASE_7d']) & (xxx['DAYS_SINCE_ADMIT']>=0)                        
    xxx['AKI0.3'] = (xxx['RESULT_NUM']-xxx['RESULT_NUM_BASE_2d']>=0.3) & (xxx['DAYS_SINCE_ADMIT']>=0)      
    
    if part == '2d':
        xxx = xxx[xxx['AKI0.3']]
        xxx = xxx.sort_values('SPECIMEN_DATE')
        xxx_backup = xxx.copy()
        xxx = xxx.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
        xxx['RESULT_NUM_BASE'] = xxx['RESULT_NUM_BASE_7d']
        #xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE'] = xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE_7d']
        
    elif part == '7d':
        xxx = xxx[xxx['AKI1.5']]
        xxx = xxx.sort_values('SPECIMEN_DATE')
        xxx_backup = xxx.copy()
        xxx = xxx.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
        xxx['RESULT_NUM_BASE'] = xxx['RESULT_NUM_BASE_7d']

    elif part == 'both':
        xxx = xxx[xxx['AKI0.3'] & xxx['AKI1.5']]
        xxx = xxx.sort_values('SPECIMEN_DATE')
        xxx_backup = xxx.copy()
        xxx = xxx.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
        xxx['RESULT_NUM_BASE'] = xxx['RESULT_NUM_BASE_7d']
        #xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE'] = xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE_7d']
        
    elif part == 'either':
        xxx = xxx[xxx['AKI0.3'] | xxx['AKI1.5']]
        xxx = xxx.sort_values(['SPECIMEN_DATE', 'RESULT_NUM'], ascending=[True, False])
        xxx_backup = xxx.copy()
        xxx = xxx.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
        xxx['RESULT_NUM_BASE'] = xxx['RESULT_NUM_BASE_7d']
        #xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE'] = xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE_7d']

    #xxx = xxx.apply(set_result_base,axis=1)
    xxx['AKI1_SINCE_ADMIT'] = xxx['DAYS_SINCE_ADMIT'].copy()
    xxx['AKI1_DATE'] = xxx['SPECIMEN_DATE'].copy()
    #xxx['AKI1_DATETIME'] = xxx['SPECIMEN_DATETIME'].copy()
    xxx['AKI1_SCR'] = xxx['RESULT_NUM'].copy()
    xxx['SCR_BASELINE'] = xxx['RESULT_NUM_BASE'].copy()
    xxx['SCR_REFERENCE'] = xxx['RESULT_NUM_BASE_2d'].copy()
    xxx['AKI1_7D'] = xxx['AKI1.5'].copy()
    xxx['AKI1_2D'] = xxx['AKI0.3'].copy()
    xxx = xxx[['PATID', 'ENCOUNTERID', 'SCR_BASELINE', 'SCR_REFERENCE',  'AKI1_DATE', 'AKI1_SCR', 'AKI1_SINCE_ADMIT', 'AKI1_7D', 'AKI1_2D']]

   
    # Check condition for AKI2: 2.0x - <3.0x
    aki2 = xxx.merge(xxx_backup, on=['PATID', 'ENCOUNTERID'], how='left')
    
    # print(aki2)
    # print(aki2['RESULT_NUM']>=2*aki2['SCR_BASELINE'])
    # print(aki2['RESULT_NUM'])
    # print(aki2['RESULT_NUM'].iloc[0])
    # print(aki2['SCR_BASELINE'].iloc[0])
    
    aki2 = aki2[aki2['SPECIMEN_DATE']>=aki2['AKI1_DATE']]
    aki2 = aki2[aki2['RESULT_NUM']>=2*aki2['SCR_BASELINE']]
    aki2 = aki2.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
    aki2['AKI2_SINCE_ADMIT'] = aki2['DAYS_SINCE_ADMIT'].copy()
    aki2['AKI2_DATE'] = aki2['SPECIMEN_DATE'].copy()
    #aki2['AKI2_DATETIME'] = aki2['SPECIMEN_DATETIME'].copy()
    aki2['AKI2_SCR'] = aki2['RESULT_NUM'].copy()
    aki2 = aki2[['PATID', 'ENCOUNTERID', 'AKI2_DATE', 'AKI2_SCR', 'AKI2_SINCE_ADMIT']]
    
    # Check condition for AKI3: SCR >= 3.0x Baseline
    aki3 = xxx.merge(xxx_backup, on=['PATID', 'ENCOUNTERID'], how='left')
    aki3 = aki3[aki3['SPECIMEN_DATE']>=aki3['AKI1_DATE']]
    aki3 = aki3[(aki3['RESULT_NUM']>=3*aki3['SCR_BASELINE']) | (aki3['RESULT_NUM']>=4)]
    aki3 = aki3.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
    aki3['AKI3_SINCE_ADMIT'] = aki3['DAYS_SINCE_ADMIT'].copy()
    aki3['AKI3_DATE'] = aki3['SPECIMEN_DATE'].copy()
    #aki3['AKI3_DATETIME'] = aki3['SPECIMEN_DATETIME'].copy()
    aki3['AKI3_SCR'] = aki3['RESULT_NUM'].copy()
    aki3 = aki3[['PATID', 'ENCOUNTERID', 'AKI3_DATE', 'AKI3_SINCE_ADMIT', 'AKI3_SCR']]
    
    # Check condition for AKI3: initiation of RRT
    #df_rrt = get_rrt(path, ext, sep, yyy)
    rrt = df_rrt.merge(xxx[['PATID', 'ENCOUNTERID', 'AKI1_DATE']], on=['PATID', 'ENCOUNTERID'], how='left')
    rrt = rrt[rrt['RRT3_ONSET_DATE'] >= rrt['AKI1_DATE']]
    aki3b =  aki3.merge(rrt, on = ['PATID', 'ENCOUNTERID'], how = 'outer')
    cond_rrt = (aki3b['RRT3_SINCE_ADMIT'] < aki3b['AKI3_SINCE_ADMIT']) | (aki3b['AKI3_SINCE_ADMIT'].isna() & aki3b['RRT3_SINCE_ADMIT'].notna())
    aki3b.loc[cond_rrt, 'AKI3_SINCE_ADMIT'] = aki3b.loc[cond_rrt, 'RRT3_SINCE_ADMIT']
    aki3b.loc[cond_rrt, 'AKI3_DATE'] = aki3b.loc[cond_rrt, 'RRT3_ONSET_DATE']
    #aki3b.loc[cond_rrt, 'AKI3_DATETIME'] = aki3b.loc[cond_rrt, 'RRT3_ONSET_DATETIME']
    
    aki3_all = aki3b[['PATID', 'ENCOUNTERID', 'AKI3_DATE', 'AKI3_SINCE_ADMIT', 'AKI3_SCR']]
    
    onset = xxx.merge(aki2, on=['PATID', 'ENCOUNTERID'], how='outer').merge(aki3_all, on=['PATID', 'ENCOUNTERID'], how='outer')
    onset = onset.merge(yyy, on=['PATID', 'ENCOUNTERID'], how='left')

    onset.columns = onset.columns.str.upper()
    onset['ONSET_DATE'] = onset['AKI1_DATE'].copy()  
    
    onset['SCR_ONSET'] = onset['AKI1_SCR'].copy() 

    onset = onset[["PATID","ENCOUNTERID", "ADMIT_DATE", 
                   'ONSET_DATE', "AKI1_SINCE_ADMIT", "AKI2_SINCE_ADMIT", 
                   "AKI3_SINCE_ADMIT", "SCR_ONSET", 
                   'SCR_BASELINE',  'SCR_REFERENCE', 'AKI1_7D', 'AKI1_2D']]

    onset['FLAG'] = (onset['AKI2_SINCE_ADMIT'].notnull()) | (onset['AKI3_SINCE_ADMIT'].notnull())
    onset['ONSET_SINCE_ADMIT'] = onset["AKI1_SINCE_ADMIT"].copy()  #onset[["AKI1_SINCE_ADMIT", "AKI2_SINCE_ADMIT", "AKI3_SINCE_ADMIT"]].min(axis=1)
        
    
    onset['AKI_STAGE'] = 0
    
    aki_s3 = onset['AKI3_SINCE_ADMIT'].notna()
    aki_s2 = onset['AKI2_SINCE_ADMIT'].notna() & onset['AKI3_SINCE_ADMIT'].isna()
    aki_s1 = onset['AKI1_SINCE_ADMIT'].notna() & onset['AKI2_SINCE_ADMIT'].isna() & onset['AKI3_SINCE_ADMIT'].isna()
    
    onset.loc[aki_s3, 'AKI_STAGE'] = 3
    onset.loc[aki_s2, 'AKI_STAGE'] = 2
    onset.loc[aki_s1, 'AKI_STAGE'] = 1
    
    return onset.drop_duplicates()

In [ ]:
onset = read_onset_new(df_onset = df_onset, df_y = df_y, df_rrt= df_rrt, df_base = df_base, part = 'either')

In [ ]:
onset[['ENCOUNTERID', 'AKI_STAGE']].groupby('AKI_STAGE').count()

In [ ]:
onset.to_pickle(pdata+'/onset00.pkl')

In [ ]:
onset[['AKI_STAGE','ENCOUNTERID']].drop_duplicates().groupby('AKI_STAGE').count().reset_index()

In [ ]:
cohort_table_df.loc['MDRD_CKD3',]+cohort_table_df.loc['MDRD_CKD4',]+cohort_table_df.loc['MDRD_CKD5',]

In [ ]:
def get_aki_onset2(df_scr, df_admit, df_rrt, df_baseline, aki_criteria = 'either'):
    xxx = df_scr.copy()
    yyy = df_admit.copy()
    ##### Filter out the CKD patients that does not have baseline 
    xxx = xxx[xxx.ENCOUNTERID.isin(df_baseline.ENCOUNTERID.unique())]  
    yyy = yyy[yyy.ENCOUNTERID.isin(df_baseline.ENCOUNTERID.unique())]  
    ######
    zzz = df_baseline[['PATID', 'ENCOUNTERID', 'SERUM_CREAT_BASE']].drop_duplicates()
    zzz.columns= ['PATID', 'ENCOUNTERID',  'RESULT_NUM_BASE_7d']
    xxx = xxx.merge(zzz, on = ['PATID', 'ENCOUNTERID'], how='left')
 
    zzz2 = xxx[['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE', 'RESULT_NUM']].groupby(['PATID', 'ENCOUNTERID']).rolling('2d', on='SPECIMEN_DATE').min().reset_index()
    zzz2 = zzz2[['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE', 'RESULT_NUM']]
    zzz2.columns= ['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE', 'RESULT_NUM_BASE_2d']
    xxx = xxx.merge(zzz2, on = ['PATID', 'ENCOUNTERID', 'SPECIMEN_DATE'], how='left')
 
    # Check condition for AKI1
    #1.5 increase in 7 days
    xxx['AKI1.5'] = (xxx['RESULT_NUM']>=1.5*xxx['RESULT_NUM_BASE_7d']) & (xxx['DAYS_SINCE_ADMIT']>=0) 
    #0.3 increase in 48 hours
    xxx['AKI0.3'] = (xxx['RESULT_NUM']-xxx['RESULT_NUM_BASE_2d']>=0.3) & (xxx['DAYS_SINCE_ADMIT']>=0)      
    if aki_criteria == '2d':
        xxx = xxx[xxx['AKI0.3']]
        xxx = xxx.sort_values(['SPECIMEN_DATE', 'RESULT_NUM'], ascending=[True, False])
        xxx_backup = xxx.copy()
        xxx = xxx.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
        xxx['RESULT_NUM_BASE'] = xxx['RESULT_NUM_BASE_7d']
        #xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE'] = xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE_7d']
    elif aki_criteria == '7d':
        xxx = xxx[xxx['AKI1.5']]
        xxx = xxx.sort_values(['SPECIMEN_DATE', 'RESULT_NUM'], ascending=[True, False])
        xxx_backup = xxx.copy()
        xxx = xxx.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
        xxx['RESULT_NUM_BASE'] = xxx['RESULT_NUM_BASE_7d']
 
    elif aki_criteria == 'both':
        xxx = xxx[xxx['AKI0.3'] & xxx['AKI1.5']]
        xxx = xxx.sort_values(['SPECIMEN_DATE', 'RESULT_NUM'], ascending=[True, False])
        xxx_backup = xxx.copy()
        xxx = xxx.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
        xxx['RESULT_NUM_BASE'] = xxx['RESULT_NUM_BASE_7d']
        #xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE'] = xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE_7d']
    elif aki_criteria == 'either':
        xxx = xxx[xxx['AKI0.3'] | xxx['AKI1.5']]
        xxx = xxx.sort_values(['SPECIMEN_DATE', 'RESULT_NUM'], ascending=[True, False])
        xxx_backup = xxx.copy()
        xxx = xxx.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
        xxx['RESULT_NUM_BASE'] = xxx['RESULT_NUM_BASE_7d']
        #xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE'] = xxx.loc[xxx['AKI1.5'], 'RESULT_NUM_BASE_7d']
 
    xxx['AKI1_SINCE_ADMIT'] = xxx['DAYS_SINCE_ADMIT'].copy()
    xxx['AKI1_DATE'] = xxx['SPECIMEN_DATE'].copy()
    xxx['AKI1_SCR'] = xxx['RESULT_NUM'].copy()
    xxx['SCR_BASELINE'] = xxx['RESULT_NUM_BASE'].copy()
    xxx['SCR_REFERENCE'] = xxx['RESULT_NUM_BASE_2d'].copy()
    xxx['AKI1_7D'] = xxx['AKI1.5'].copy()
    xxx['AKI1_2D'] = xxx['AKI0.3'].copy()
    xxx = xxx[['PATID', 'ENCOUNTERID', 'SCR_BASELINE', 'SCR_REFERENCE',  'AKI1_DATE', 'AKI1_SCR', 'AKI1_SINCE_ADMIT', 'AKI1_7D', 'AKI1_2D']]
 
    # Check condition for AKI2: 2.0x - <3.0x
    aki2 = xxx.merge(xxx_backup, on=['PATID', 'ENCOUNTERID'], how='left')
    aki2 = aki2[aki2['SPECIMEN_DATE']>=aki2['AKI1_DATE']]
    aki2 = aki2[aki2['RESULT_NUM']>=2*aki2['SCR_BASELINE']]
    aki2 = aki2.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
    aki2['AKI2_SINCE_ADMIT'] = aki2['DAYS_SINCE_ADMIT'].copy()
    aki2['AKI2_DATE'] = aki2['SPECIMEN_DATE'].copy()
    aki2['AKI2_SCR'] = aki2['RESULT_NUM'].copy()
    aki2 = aki2[['PATID', 'ENCOUNTERID', 'AKI2_DATE', 'AKI2_SCR', 'AKI2_SINCE_ADMIT']]
    # Check condition for AKI3: SCR >= 3.0x Baseline
    aki3 = xxx.merge(xxx_backup, on=['PATID', 'ENCOUNTERID'], how='left')
    aki3 = aki3[aki3['SPECIMEN_DATE']>=aki3['AKI1_DATE']]
    aki3 = aki3[(aki3['RESULT_NUM']>=3*aki3['SCR_BASELINE']) | (aki3['RESULT_NUM']>=4)]
    aki3 = aki3.groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
    aki3['AKI3_SINCE_ADMIT'] = aki3['DAYS_SINCE_ADMIT'].copy()
    aki3['AKI3_DATE'] = aki3['SPECIMEN_DATE'].copy()
    aki3['AKI3_SCR'] = aki3['RESULT_NUM'].copy()
    aki3 = aki3[['PATID', 'ENCOUNTERID', 'AKI3_DATE', 'AKI3_SINCE_ADMIT', 'AKI3_SCR']]
    # Check condition for AKI3: initiation of RRT
    #df_rrt = get_rrt(path, ext, sep, yyy)
    rrt = df_rrt.merge(xxx[['PATID', 'ENCOUNTERID', 'AKI1_DATE']], on=['PATID', 'ENCOUNTERID'], how='left')
    rrt = rrt[rrt['RRT3_ONSET_DATE'] >= rrt['AKI1_DATE']]
    aki3b =  aki3.merge(rrt, on = ['PATID', 'ENCOUNTERID'], how = 'outer')
    cond_rrt = (aki3b['RRT3_SINCE_ADMIT'] < aki3b['AKI3_SINCE_ADMIT']) | (aki3b['AKI3_SINCE_ADMIT'].isna() & aki3b['RRT3_SINCE_ADMIT'].notna())
    aki3b.loc[cond_rrt, 'AKI3_SINCE_ADMIT'] = aki3b.loc[cond_rrt, 'RRT3_SINCE_ADMIT']
    aki3b.loc[cond_rrt, 'AKI3_DATE'] = aki3b.loc[cond_rrt, 'RRT3_ONSET_DATE']
    aki3_all = aki3b[['PATID', 'ENCOUNTERID', 'AKI3_DATE', 'AKI3_SINCE_ADMIT', 'AKI3_SCR']]
    # Merge AKI staging information
    onset = xxx.merge(aki2, on=['PATID', 'ENCOUNTERID'], how='outer').merge(aki3_all, on=['PATID', 'ENCOUNTERID'], how='outer')
    onset = onset.merge(yyy, on=['PATID', 'ENCOUNTERID'], how='left')
 
    onset.columns = onset.columns.str.upper()
    onset['ONSET_DATE'] = onset['AKI1_DATE'].copy()  
    onset['SCR_ONSET'] = onset['AKI1_SCR'].copy() 
#    onset['DISCHARGE_SINCE_ONSET'] = (onset['DISCHARGE_DATE'] - onset['ONSET_DATE']).dt.days
    onset['DISCHARGE_SINCE_ONSET'] = 0
    onset['DISCHARGE_DATE'] = 0    
    onset = onset[['PATID','ENCOUNTERID', 'ADMIT_DATE', 'DISCHARGE_DATE', 
                   'ONSET_DATE', 'AKI1_SINCE_ADMIT', 'AKI2_SINCE_ADMIT', 
                   'AKI3_SINCE_ADMIT',  'DISCHARGE_SINCE_ONSET','SCR_ONSET', 
                   'SCR_BASELINE',  'SCR_REFERENCE', 'AKI1_7D', 'AKI1_2D']]
 
    onset['FLAG'] = (onset['AKI2_SINCE_ADMIT'].notna()) | (onset['AKI3_SINCE_ADMIT'].notna())
    onset['ONSET_SINCE_ADMIT'] = onset['AKI1_SINCE_ADMIT'].copy()  #onset[['AKI1_SINCE_ADMIT', 'AKI2_SINCE_ADMIT', 'AKI3_SINCE_ADMIT']].min(axis=1)
    #Generate onset staging by taking the first stage
    onset['AKI_STAGE'] = 0
    filter_aki3 = onset['AKI3_SINCE_ADMIT'].notna()
    filter_aki2 = onset['AKI2_SINCE_ADMIT'].notna() & onset['AKI3_SINCE_ADMIT'].isna()
    filter_aki1 = onset['AKI1_SINCE_ADMIT'].notna() & onset['AKI2_SINCE_ADMIT'].isna() & onset['AKI3_SINCE_ADMIT'].isna()
    onset.loc[filter_aki3, 'AKI_STAGE'] = 3
    onset.loc[filter_aki2, 'AKI_STAGE'] = 2
    onset.loc[filter_aki1, 'AKI_STAGE'] = 1

    # Determine the initial AKI stage by finding the column with the smallest onset day
    #onset['AKI_INIT_STG'] = onset.apply(determine_initial_aki_stage, axis=1)
    #onset3 = onset2.sort_values('ONSET_SINCE_ADMIT').groupby(['PATID', 'ENCOUNTERID']).first().reset_index()
    return onset.drop_duplicates()

In [ ]:
onset2 = get_aki_onset2(df_scr = df_onset, df_admit = df_y, df_rrt = df_rrt, df_baseline = df_base, aki_criteria = 'either')

In [ ]:
onset2[['AKI_STAGE','ENCOUNTERID']].drop_duplicates().groupby('AKI_STAGE').count().reset_index()